# IO Operations and Statistical Method

In [182]:
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)
import pandas as pd

In [183]:
def df_info(df: pd.DataFrame) -> None:
    return df.head(n=20).style

## File Operations

In [184]:
cols = [
    'Survived', 'Pclass', 'Sex', 'Age', 'SibSp',
    'Parch', 'Fare', 'Cabin', 'Embarked'
]

df = pd.read_csv("../data/titanic/dataset.csv")
print(df.columns)

df = pd.DataFrame(df[cols], index=df["PassengerId"])
df.index.name = 'ID'

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [185]:
df_info(df)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
ID,,,,,,,,,
1,1.000000,1.000000,female,38.000000,1.000000,0.000000,71.283300,C85,C
2,1.000000,3.000000,female,26.000000,0.000000,0.000000,7.925000,nan,S
3,1.000000,1.000000,female,35.000000,1.000000,0.000000,53.100000,C123,S
4,0.000000,3.000000,male,35.000000,0.000000,0.000000,8.050000,nan,S
5,0.000000,3.000000,male,nan,0.000000,0.000000,8.458300,nan,Q
6,0.000000,1.000000,male,54.000000,0.000000,0.000000,51.862500,E46,S
7,0.000000,3.000000,male,2.000000,3.000000,1.000000,21.075000,nan,S
8,1.000000,3.000000,female,27.000000,0.000000,2.000000,11.133300,nan,S
9,1.000000,2.000000,female,14.000000,1.000000,0.000000,30.070800,nan,C


In [186]:
df.to_csv("../data/titanic/modified_dataset.csv")

### Statistical Functions

In [187]:
print(df["Age"].cov(s2))

-5.919861369328288


In [188]:
print(df["Age"].corr(s2))

-0.9803265963228514


In [192]:
df_info(df.describe())

,Survived,Pclass,Age,SibSp,Parch,Fare
count,890.000000,890.000000,713.000000,890.000000,890.000000,890.000000
mean,0.384270,2.307865,29.709916,0.522472,0.382022,32.232246
std,0.486696,0.836220,14.533827,1.103247,0.806409,49.714317
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.000000,0.000000,0.000000,7.925000
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [190]:
df.agg(["min", "max", "median", "mean", "var", "std"])

,Survived,Pclass,Age,SibSp,Parch,Fare
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200
median,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
mean,0.384270,2.307865,29.709916,0.522472,0.382022,32.232246
var,0.236873,0.699263,211.232130,1.217155,0.650295,2471.513364
std,0.486696,0.836220,14.533827,1.103247,0.806409,49.714317


In [191]:
print(df["Survived"].value_counts())

0.0    548
1.0    342
Name: Survived, dtype: int64
